In [2]:
from IPython.display import HTML, display

display(HTML('''
    <script>
        code_show=true; 
        function code_toggle() {
            if (code_show){
                $('div.input').hide();
            } else {
                $('div.input').show();
            }
            code_show = !code_show;
        } 
        $(document).ready(function() {
            code_toggle(); // Hide code on load

            // Expand the notebook output area to prevent scrollbars
            $('.output_wrapper, .output').css({
                'max-height': 'none',
                'height': 'auto',
                'overflow-y': 'visible'
            });

            // Also try expanding the main container if needed
            $('.container').css('width', '100%');
        });
    </script>
    <button onclick="code_toggle()">▶ Click me to toggle code</button>
'''))



import pandas as pd
from datetime import datetime
from ipywidgets import VBox, HBox, Button, Text, DatePicker, Checkbox, Dropdown, Label, TimePicker, Output
from ipywidgets import HTML as WidgetHTML
from IPython.display import HTML
from functools import partial
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from datetime import timedelta


# File to store tasks
TASKS_FILE = "tasks.csv"
# File to store trashed tasks
TRASH_FILE = "trash.csv"


# Load existing tasks or create new
def load_tasks():
    try:
        return pd.read_csv(TASKS_FILE)
    except FileNotFoundError:
        return pd.DataFrame(columns=["Completed", "Task", "Company", "Deadline"])
    
# Save tasks
def save_tasks(df):
    df.to_csv(TASKS_FILE, index=False)
    
# Move to trash
def move_to_trash(row):
    try:
        trash_df = pd.read_csv("trash.csv")
    except FileNotFoundError:
        trash_df = pd.DataFrame(columns=["Completed", "Task", "Company", "Deadline"])

    # Convert the row (Series) to a one-row DataFrame with correct columns
    row_df = pd.DataFrame([{
        "Completed": row["Completed"],
        "Task": row["Task"],
        "Company": row["Company"],
        "Deadline": row["Deadline"]
    }])
    
    trash_df = pd.concat([trash_df, row_df], ignore_index=True)
    trash_df.to_csv("trash.csv", index=False)

    
# Trash tasks 
def trash_task(b, idx):
    global tasks_df
    row = tasks_df.loc[idx]
    move_to_trash(row)
    tasks_df = tasks_df.drop(index=idx).reset_index(drop=True)
    save_tasks(tasks_df)
    display_tasks()
    
# Complete tasks
def mark_complete(b, idx):
    global tasks_df
    tasks_df.at[idx, "Completed"] = True
    save_tasks(tasks_df)
    display_tasks()
    
# Create widgets for task entry
task_input = Text(description="Task:")
company_input = Text(description="Company:")
date_input = DatePicker(description="Date:")
time_input = Text(description="Deadline:", placeholder="e.g. 14:30")
add_button = Button(description="Add Task", button_style="success")
calendar_checkbox = Checkbox(description="Add to Google Calendar", indent=False)
reminder_offset = Text(
    description="Remind:",
    placeholder="1d = 1 day, 2h, 30m, 1w...",
    layout={"width": "450px"}
)

output = Output()

# Load and show task list
tasks_df = load_tasks()

# Controls for filtering and sorting
show_completed = True
sort_column = "Deadline"
sort_ascending = True

# Define toggle completion
def toggle_completion(change, idx):
   tasks_df.at[idx, "Completed"] = change["new"]
   save_tasks(tasks_df)
    
# Define toggle completion
def toggle_complete(b, idx):
    global tasks_df
    current_status = tasks_df.at[idx, "Completed"]
    tasks_df.at[idx, "Completed"] = not current_status
    save_tasks(tasks_df)
    display_tasks()

def display_tasks():
    global tasks_df
    output.clear_output()
    with output:
        if tasks_df.empty:
            print("No tasks yet.")
        else:
            df = tasks_df.copy()
            df["Deadline"] = pd.to_datetime(df["Deadline"], errors="coerce")

            # Apply filters
            if not show_completed:
                df = df[df["Completed"] == False]

            # Sort
            df = df.sort_values(by=sort_column, ascending=sort_ascending).reset_index(drop=True)

            # Header row with sorting buttons
            header = HBox([
                Button(description="Task", layout={"width": "250px"}, button_style=""),
                Button(description="Company", layout={"width": "150px"}, button_style=""),
                Button(description="Deadline", layout={"width": "140px"}, button_style=""),
                Label("Delete", layout={"width": "80px"}),
                Label("Toggle", layout={"width": "90px"})
            ])
            header.children[0].on_click(set_sort("Task"))
            header.children[1].on_click(set_sort("Company"))
            header.children[2].on_click(set_sort("Deadline"))

            rows = []
            for idx, row in df.iterrows():
                is_completed = row["Completed"]
                task_display = f"✓ {row['Task']}" if is_completed else row["Task"]

                deadline_dt = pd.to_datetime(row["Deadline"])
                deadline_str = deadline_dt.strftime("%m-%d %I:%M %p")
                is_overdue = not is_completed and deadline_dt < datetime.now()

                # Style if overdue
                task_style = "color: red; font-weight: bold;" if is_overdue else ""
                company_style = "color: red; font-weight: bold;" if is_overdue else ""
                deadline_style = "color: red; font-weight: bold;" if is_overdue else ""
        
                task_label = WidgetHTML(f"<div style='width:250px; {task_style}'>{task_display}</div>")
                company_label = WidgetHTML(f"<div style='width:150px; {company_style}'>{row['Company']}</div>")
                deadline_label = WidgetHTML(f"<div style='width:140px; {deadline_style}'>{deadline_str}</div>")

                trash_button = Button(description="Trash", button_style="danger", layout={"width": "70px"})
                trash_button.on_click(partial(trash_task, idx=idx))



                toggle_button = Button(
                    description="Undo" if is_completed else "Complete",
                    button_style="warning" if is_completed else "success",
                    layout={"width": "80px"}
                )
                toggle_button.on_click(partial(toggle_complete, idx=tasks_df.index[tasks_df["Task"] == row["Task"]].tolist()[0]))

                row_box = HBox([
                    task_label,
                    company_label,
                    deadline_label,
                    trash_button,
                    toggle_button
                ])
                rows.append(row_box)

            display(VBox([header] + rows + [toggle_button_display]))
            
# Add task handler
def on_add_clicked(b):
    task = task_input.value
    company = company_input.value
    date = date_input.value
    time_str = time_input.value
    if not all([task, company, date, time_str]):
        print("Please fill in all fields.")
        return

    try:
        deadline = f"{date.strftime('%Y-%m-%d')} {time_str}"
        pd.to_datetime(deadline)  # Validate datetime format
        tasks_df.loc[len(tasks_df)] = [False, task, company, deadline]
        save_tasks(tasks_df)

        SCOPES = ["https://www.googleapis.com/auth/calendar.events"]

        def get_credentials():
            creds = None
            if os.path.exists("token.json"):
                creds = Credentials.from_authorized_user_file("token.json", SCOPES)
            # If no (valid) credentials, do login flow
            if not creds or not creds.valid:
                if creds and creds.expired and creds.refresh_token:
                    creds.refresh(Request())
                else:
                    flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
                    creds = flow.run_local_server(port=0)
                # Save credentials for next time
                with open("token.json", "w") as token:
                    token.write(creds.to_json())
            return creds

        def add_event_to_calendar(task, company, deadline, reminder=None):
            creds = get_credentials()
            service = build("calendar", "v3", credentials=creds)

            start_time = pd.to_datetime(deadline)
            end_time = start_time + timedelta(minutes=30)

            event = {
                "summary": f"{task} [{company}]",
                "start": {
                    "dateTime": start_time.isoformat(),
                    "timeZone": "America/New_York"  # Adjust as needed
                },
                "end": {
                    "dateTime": end_time.isoformat(),
                    "timeZone": "America/New_York"
                },
                "description": "Task from Jupyter Task Manager"
            }

            if reminder is not None:
                event["reminders"] = {
                    "useDefault": False,
                    "overrides": [{"method": "popup", "minutes": int(reminder)}]
                }


            service.events().insert(calendarId="primary", body=event).execute()
            print(f"📆 Task added to Google Calendar with reminder ({reminder} mins before).")
        def parse_reminder_offset(offset_str):
            import re
            total_minutes = 0
            pattern = re.findall(r"(\d+)([mhwd])", offset_str.lower())
            for value, unit in pattern:
                value = int(value)
                if unit == "m":
                    total_minutes += value
                elif unit == "h":
                    total_minutes += value * 60
                elif unit == "d":
                    total_minutes += value * 60 * 24
                elif unit == "w":
                    total_minutes += value * 60 * 24 * 7
                # Months are tricky — you could approximate to 30 days:
                # elif unit == "mo":
                #     total_minutes += value * 60 * 24 * 30
            return total_minutes

        if calendar_checkbox.value:
            try:
                try:
                    reminder_minutes = parse_reminder_offset(reminder_offset.value) if reminder_offset.value else None
                except Exception as e:
                    print(f"⚠️ Invalid reminder format: {e}")
                    reminder_minutes = None

                add_event_to_calendar(task, company, deadline, reminder=reminder_minutes)
                
                # Parse reminder offset before calling add_event_to_calendar
                reminder_minutes = None
                if reminder_offset.value:
                    try:
                        reminder_minutes = parse_reminder_offset(reminder_offset.value)
                        deadline_dt = pd.to_datetime(deadline)
                        reminder_time = deadline_dt - timedelta(minutes=reminder_minutes)

                        if reminder_time < datetime.now():
                            print("⚠️ Reminder is set in the past! Adjust the time before submitting.")
                            return
                    except Exception as e:
                        print(f"⚠️ Invalid reminder format: {e}")
                        return


            except Exception as e:
                print(f"⚠️ Could not add to Google Calendar: {e}")

        task_input.value = company_input.value = time_input.value = ''
        display_tasks()

    except Exception as e:
        print(f"Error with deadline format: {e}")

from calendar_utils import add_event_to_calendar
add_button.on_click(on_add_clicked)

# UI layout
display(VBox([
    Label("Add a Task:"),
    HBox([task_input, company_input]),
    HBox([date_input, time_input]),
    HBox([calendar_checkbox, reminder_offset]),
    add_button,
    output
]))

# Toggle show/hide completed
def toggle_show_completed(b):
    global show_completed
    show_completed = not show_completed
    b.description = "Show All Tasks" if not show_completed else "Hide Completed Tasks"
    display_tasks()

toggle_button_display = Button(
    description="Hide Completed Tasks",
    button_style="info",
    layout={"margin": "10px 0px 10px 0px"}
)
toggle_button_display.on_click(toggle_show_completed)

# Sorting button handlers
def set_sort(col):
    def handler(b):
        global sort_column, sort_ascending
        if sort_column == col:
            sort_ascending = not sort_ascending  # toggle direction
        else:
            sort_column = col
            sort_ascending = True
        display_tasks()
    return handler


# Show initial tasks
display_tasks()



📆 Task added to Google Calendar with reminder (0 mins before).
